In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import math
import datetime
%load_ext tensorboard
(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

ModuleNotFoundError: No module named 'tensorflow_datasets'

1) a + b >= 5 </br>
single binary output

2) y = a - b </br>
integer in range [-9,9]
One-hots with 18 values?

In [5]:
# building the data pipeline for subtask one (a + b >= 5)

def prepare_subtask_1(mnist):
    # flatten the images into vectors
    mnist = mnist.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    # convert data from uint8 to float32
    mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # bringing image values from range [0,255] to [-1,1]
    mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
    # zip two random parts of the dataset result will be a tuple of tuples: ((i1,t1),(i2,t2))
    mnist = tf.data.Dataset.zip((mnist.shuffle(2000),mnist.shuffle(2000)))
    # flatten the nested tuples into shape: (i1,i2,t) and calculate the new target
    mnist = mnist.map(lambda d1, d2: (d1[0],d2[0],d1[1] + d2[1] >= 5))
    # cache progress in memory
    mnist = mnist.cache()
    # shuffle, batch, prefetch
    mnist = mnist.shuffle(1000)
    mnist = mnist.batch(32)
    mnist = mnist.prefetch(20)
    
    # return processed dataset
    return mnist

In [6]:
# building the data pipeline for subtask 2 (y = a - b)

def prepare_subtask_2(mnist):
    # flatten the images into vectors
    mnist = mnist.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    # convert data from uint8 to float32
    mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # bringing image values from range [0,255] to [-1,1]
    mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
    # zip two random parts of the dataset result will be a tuple of tuples: ((i1,t1),(i2,t2))
    mnist = tf.data.Dataset.zip((mnist.shuffle(2000),mnist.shuffle(2000)))
    # flatten the nested tuples into shape: (i1,i2,t) and calculate new target
    mnist = mnist.map(lambda d1, d2: (d1[0],d2[0],d1[1] - d2[1]))
    # create one-hot targets with depth 19 to accomodate all values between -9 and 9
    # since 0 - 9 and 9 - 0 are the smallest and biggest possible target
    mnist = mnist.map(lambda img1, img2, target: (img1, img2, tf.one_hot(target, depth=19)))
    # cache progress in memory
    mnist = mnist.cache()
    # shuffle, batch, prefetch
    mnist = mnist.shuffle(1000)
    mnist = mnist.batch(32)
    mnist = mnist.prefetch(20)
    # return processed dataset
    return mnist

In [7]:
# Model for Subtask 1 (a + b >= 5)
class ModelOne(tf.keras.Model):
    def __init__(self, optimizer):
        super().__init__()
        
        # optimizer as passable parameter so we can run different tests with SGD and Adam
        self.optimizer = optimizer
        
        # some metrics
        self.metrics_list = [
            tf.keras.metrics.Mean(name="loss"),
            tf.keras.metrics.BinaryAccuracy(name="acc")    
        ]
        
        # binary loss function as output can either be 0 or 1
        self.loss_function = tf.keras.losses.BinaryCrossentropy()
        
        # define layers
        # layer 1 to 3 are bascially the same network architecture as last week to decode the number
        # layer 4 will do the combination of both
        # output will be a single probability 
        self.layer1 = tf.keras.layers.Dense(100, activation="relu")
        self.layer2 = tf.keras.layers.Dense(100, activation="relu")
        self.layer3 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        self.layer4 = tf.keras.layers.Dense(100, activation="relu")
        self.output_unit = tf.keras.layers.Dense(1, activation=tf.nn.softmax)
    
    # reuse the first layers for both inputs
    def call(self, x, training=False):
        img1, img2 = x
        # get activation for the first digit
        digit1 = self.layer1(img1)
        digit1 = self.layer2(digit1)
        digit1 = self.layer3(digit1)
        
        # get activation for the second digit
        digit2 = self.layer1(img2)
        digit2 = self.layer2(digit2)
        digit2 = self.layer3(digit2)
        
        # add outputs together
        #joined = digit1 + digit2
        joined = tf.concat([digit1, digit2], axis = 1)
        
        # get activation for the combination
        out = self.layer4(joined)
        out = self.output_unit(out)
        
        return out
    
    # reset all collected metrics
    def reset_metrics(self):
        
        for metric in self.metrics:
            metric.reset_states()
            
    
    # define train step
    @tf.function
    def train_step(self, data):
        # I have a triple instead of tuple, so I have to split it into three parts
        x1, x2, targets = data
        # and add the first two to a seperate tuple again, to feed it into the network
        x = (x1,x2)
        
        # get gradients and apply them to the optimizer
        with tf.GradientTape() as tape:
            predictions = self(x, training=True)
            loss = self.loss_function(targets,predictions)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update loss metric
        self.metrics[0].update_state(loss)
        
        # for all metrics except loss, update states (in this case only accuracy)
        self.metrics[1].update_state(targets,predictions)
            
        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
    
    @tf.function
    def test_step(self, data):
        # prepare data in the same way as above
        x1, x2, targets = data
        x = (x1,x2)
        predictions = self(x, training=False)
        loss = self.loss_function(targets, predictions)
        
        # update metrics
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(targets, predictions)
            
        return {m.name: m.result() for m in self.metrics}

In [14]:
# Model for Subtask 1 (a + b >= 5)
class ModelTwo(tf.keras.Model):
    def __init__(self, optimizer):
        super().__init__()
        
        # optimizer as passable parameter so we can run different tests with SGD and Adam
        self.optimizer = optimizer
        
        # some metrics
        self.metrics_list = [
            tf.keras.metrics.Mean(name="loss"),
            tf.keras.metrics.BinaryAccuracy(name="acc")    
        ]
        
        # categorical loss function as we have one-hots again
        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        
        # define layers
        # layer 1 to 3 are bascially the same network architecture as last week to decode the number
        # layer 4 will do the combination of both
        self.layer1 = tf.keras.layers.Dense(100, activation="relu")
        self.layer2 = tf.keras.layers.Dense(100, activation="relu")
        self.layer3 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        self.layer4 = tf.keras.layers.Dense(100, activation="relu")
        self.output_unit = tf.keras.layers.Dense(19, activation=tf.nn.softmax)
    
    # reuse the first layers for both inputs
    def call(self, x, training=False):
        img1, img2 = x
        # get activation for the first digit
        digit1 = self.layer1(img1)
        digit1 = self.layer2(digit1)
        digit1 = self.layer3(digit1)
        
        # get activation for the second digit
        digit2 = self.layer1(img2)
        digit2 = self.layer2(digit2)
        digit2 = self.layer3(digit2)
        
        # add outputs together
        #joined = digit1 + digit2
        joined = tf.concat([digit1, digit2], axis = 1)
        
        # get activation for the combination
        out = self.layer4(joined)
        out = self.output_unit(out)
        
        return out
    
    # reset all collected metrics
    def reset_metrics(self):
        
        for metric in self.metrics:
            metric.reset_states()
            
    
    # define train step
    @tf.function
    def train_step(self, data):
        # I have a triple instead of tuple, so I have to split it into three parts
        x1, x2, targets = data
        # and add the first two to a seperate tuple again, to feed it into the network
        x = (x1,x2)
        
        # get gradients and apply them to the optimizer
        with tf.GradientTape() as tape:
            predictions = self(x, training=True)
            loss = self.loss_function(targets,predictions)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update loss metric
        self.metrics[0].update_state(loss)
        
        # for all metrics except loss, update states (in this case only accuracy)
        self.metrics[1].update_state(targets,predictions)
            
        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
    
    @tf.function
    def test_step(self, data):
        # prepare data in the same way as above
        x1, x2, targets = data
        x = (x1,x2)
        predictions = self(x, training=False)
        loss = self.loss_function(targets, predictions)
        
        # update metrics
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(targets, predictions)
            
        return {m.name: m.result() for m in self.metrics}

In [10]:
import pprint
import tqdm

def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer):
    for epoch in range(epochs):
        print(f"Epoch {epoch}:")
        
        # Training:
        
        for data in tqdm.tqdm(train_ds, position=0, leave=True):
            metrics = model.train_step(data)
            
            # logging the validation metrics to the log file which is used by tensorboard
            with train_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics (requires a reset_metrics method in the model)
        model.reset_metrics()    
        
        # Validation:
        for data in val_ds:
            metrics = model.test_step(data)
        
            # logging the validation metrics to the log file which is used by tensorboard
            with val_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
                    
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics
        model.reset_metrics()
        print("\n")

In [11]:
%tensorboard --logdir logs/

In [34]:
# complete training with instanciating everything necessary beforhand
# subtask is indicated by a number (1 or 2)
def automated_training(subtask,optimizer,epochs):
    if subtask == 1:
        model = ModelOne(optimizer)
        train_data = train_ds.apply(prepare_subtask_1)
        test_data = test_ds.apply(prepare_subtask_1)
    elif subtask == 2:
        model = ModelTwo(optimizer)
        train_data = train_ds.apply(prepare_subtask_2)
        test_data = test_ds.apply(prepare_subtask_2)
    else:
        print("Not a valid input.")
        return
    
    # get a handle for the optimizer to print to the log
    # I know this is not the nicest way to do this...
    if optimizer == type(tf.keras.optimizers.SGD()):
        name = "SGD"
    elif optimizer == type(tf.keras.optimizers.Adam()):
        name = "Adam"
    else:
        name = "other"
    
    # Define where to save the log
    config_name= "config_name"
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train/task{str(subtask)}/{name}"
    val_log_path = f"logs/{config_name}/{current_time}/val/task{str(subtask)}/{name}"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    # run the training loop 
    training_loop(model=model, 
            train_ds=train_data, 
            val_ds=test_data, 
            epochs=epochs, 
            train_summary_writer=train_summary_writer, 
            val_summary_writer=val_summary_writer)

In [33]:
op = tf.keras.optimizers.SGD()
type(op) == type(tf.keras.optimizers.SGD())

True

In [16]:
# run training with SGD and Adam
# first subtask 1
automated_training(1,tf.keras.optimizers.SGD(),10)
automated_training(1,tf.keras.optimizers.Adam(),10)
# subtask 2
automated_training(1,tf.keras.optimizers.SGD(),10)
automated_training(2,tf.keras.optimizers.Adam(),10)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 0:


...

['loss: 1.2574186325073242', 'acc: 0.9710149168968201']
['val_loss: 1.2220274209976196', 'val_acc: 0.971184492111206']


Epoch 8:


100%|██████████████████████████████████████| 1875/1875 [00:02<00:00, 682.84it/s]


['loss: 1.2660272121429443', 'acc: 0.9709717631340027']
['val_loss: 1.2538138628005981', 'val_acc: 0.9711847901344299']


Epoch 9:


100%|██████████████████████████████████████| 1875/1875 [00:02<00:00, 685.64it/s]


['loss: 1.267561435699463', 'acc: 0.9709891676902771']
['val_loss: 1.3715094327926636', 'val_acc: 0.9711838960647583']


